<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/potencia_inv_def.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Metodo potencia inversa con deflacion.
import numpy as np
import math
def min_eigen(A,tol,max_iter,pr=True,v_0=[]):
    n=len(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/math.sqrt(n))#Normalizado
    L,U=factorizar_LU(A)
    v_1=solve(L,U,v_0)#Se resuelve el sistema
    lambda_0=np.dot(v_0,v_1)
    lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
    count=0
    while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
        count+=1
        v_0=v_1/np.linalg.norm(v_1)#Se normaliza
        v_1=solve(L_m=L,U_m=U,b=v_0)#iteracion
        lambda_0=lambda_1
        lambda_1=np.dot(v_0,v_1)
    if(count>=max_iter):
        print("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("numero de iteraciones: "+str(count))
    return([1/lambda_1,v_1])
def deflacion(A,l,v):
    for i in range(len(A)):
        for j in range(len(A[0])):
            A[i][j]-=l*v[i]*v[j]
    return(A)
import numpy as np
def factorizar_LU(matriz):#Codigo reciclado: Factorizacion LU
    n=len(matriz)
    L=np.zeros((n,n))
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            sum=0
            for k in range(i):
                sum+=L[i][k]*U[k][j]
            L[i][j]=matriz[i][j]-sum
            if(i>j):
                sum=0
                for k in range(j):
                    sum+=L[j][k]*U[k][i]
                U[j][i]=(matriz[j][i]-sum)/L[j][j]
            else: 
                U[i][j]=1 
                if(L[i][i]==0):
                    return("Error en la construcción de L")
    return([L,U])

def superior(list,b):#Codigo reciclado:Resolver sistema triangular
  n=len(list)
  X=[b[n-1]/list[n-1][n-1]]
  X_inv=[X[0]]
  for i in range(1,n):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[n-i-1][n-1-j]
    X.append((b[n-1-i]-acum)/list[n-1-i][n-1-i])
    X_inv.insert(0,X[i])
  return X_inv

#Codigo reciclado:Resolver sistema triangular
def inferior(list,b):#list debe contener las entradas de la matriz triangular.
#Será aceptable introducir un arreglo de arreglos de distintos tamaños, para solo introducir las entradas no cero.
  X=[b[0]/list[0][0]]
  for i in range(1,len(list)):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[i][j]
    X.append((b[i]-acum)/list[i][i])
  return X
def solve(L_m,U_m,b):
    y=inferior(L_m,b)#Resolver Ly=b
    return(superior(U_m,y))#Resolver Ux=y

def entrada_max(A):#Regresa una lista [i,j], tal que a_{i,j} es el valor maximo en valor absoluto.
#Pensado para matrices simetricas
    max=abs(A[0][1])
    index=[0,1]
    for i in range(len(A)):
        for j in range(i,len(A[0])):
            if(i!=j):
                if(max<abs(A[i][j])):
                    max=abs(A[i][j])
                    index=i,j
    return(index)
import numpy as np

In [29]:
def multiple_min_eigen(A,m,tol=10**(-8),max_iter=4000):
    eigenvectors=[]
    eigenvalues=[]
    v_0_g=np.array([1]*len(A))/np.sqrt(len(A))
    for j in range(m):
        count=0
        v_0=v_0_g
        if (j==0):
            initial=min_eigen(A,tol=10**(-18),max_iter=1000,pr=False)
            eigenvalues.append(1/initial[0])
            eigenvectors.append(initial[1])
            
        else:
            val_aux = tol + 1
            val_min = 0
            while(abs(val_aux-val_min)>tol ):
                aux=v_0
                count+=1
                for k in range(j):#quitar contrib
                    v_0-=np.dot(aux,eigenvectors[k])*np.array(eigenvectors[k])
                v_i_next=np.linalg.solve(A,v_0)
                val_min = val_aux
                val_aux = np.dot(v_0, v_0)/np.dot(v_0, v_i_next)
                v_0=(v_i_next/np.linalg.norm(v_i_next))
                
            print(count)
            eigenvalues.append(1/val_aux)
            eigenvectors.append(v_i_next)
        print(1/eigenvalues[j])
        print(eigenvectors[j])
def potencia_inv_def(A, M, tol, k):
    n = len(A)
    x0 = [1]*n
    values = np.zeros(k) #Inicializa vector de k eigenvalores 
    vectors = [] #Inicializa lista de k eigenvectores 
    for i in range(1, k+1):
        val_aux = tol + 1
        val_min = 0
        cont = 0  #Inicializa contador de iteraciones
        vec_min = x0
        while cont < M and tol < np.abs(val_aux - val_min):
            #Quitamos la contribución del menor valor propio
            for j in range(n-i+1, n): 
                vec_min -= np.array(np.dot(vectors[n-j-1], vec_min))*vectors[n-j-1]
            #Aplicamos el método de la potencia para el vec_min calculado
            norma = np.linalg.norm(vec_min)
            vec_min = vec_min/norma
            vec_aux = np.linalg.solve(A,vec_min)
            val_min = val_aux
            val_aux = np.dot(vec_min, vec_min)/np.dot(vec_aux, vec_min)
            vec_min = vec_aux
            #Revisa si se ha alcanzdo al tolerancia
            if np.abs(val_min - val_aux) < tol: 
                norma = np.linalg.norm(vec_min)
                vec_min = vec_min/norma
                break
            cont = cont + 1 
        #Guarda las listas de valores y vectores propios calculados
        values[i-1]=val_aux
        vectors += [vec_min] 
        print(values[i-1])
    return [values,vectors]


In [30]:
A_1=np.loadtxt("Eigen_1000x1000.txt",skiprows=1)
potencia_inv_def(A_1,M=4000,tol=10**(-4),k=450)

<ipython-input-29-00758e7edf99>:66: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "potencia_inv_def_numba" failed type inference due to: No implementation of function Function(<function norm at 0x7fbaca0ad950>) found for signature:
 
 >>> norm(list(int64)<iv=None>)
 
There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Overload in function 'norm_impl': File: numba/np/linalg.py: Line 2352.
        With argument(s): '(list(int64)<iv=None>)':
       Rejected as the implementation raised a specific error:
         TypingError: np.linalg.norm() only supported for array types 
  raised from /usr/local/lib/python3.7/dist-packages/numba/np/linalg.py:892

During: resolving callee type: Function(<function norm at 0x7fbaca0ad950>)
During: typing of call at <ipython-input-29-00758e7edf99> (83)


File "<ipython-input-29-00758e7edf99>", line 83:
def potencia_inv_def_numba(A, M, tol, k):
    <source elided>
       

9.99707511860496
19.99853684009322
29.998403512117125
39.9979020302518
49.99830471626898
59.99922874954007
69.99834823474681
79.99798156642575
89.99728677902121
99.99964736036675
109.99801307007296
119.99920145438924
129.9986982282731
139.99785591661168
149.99882196699426
159.9997727580885


KeyboardInterrupt: ignored

In [21]:
from numba import njit
import time
from datetime import timedelta